[프로세스]

1. 하나의 tourney 대상으로 메타 분석
    play.limitlesstcg.com/tournament/6779bb0bed2c78098c6b086b/metagame

    step a) 아키타입 별 상대 승률
    step b) 20장 덱 유형 별 vs 아키타입 승률
    step c) 아키타입 별 승률 높은 20장 덱 유형 분석 (궁극의 덱이 무엇일까)

2. 전체 tourney 묶어서 다시 분석

    step a) Standard한 Tourney 분류
            포맷을 Standard로 설정해도, 스탠다드 룰이 아닌 Tourney가 있음. 
            

# Base

In [1]:
### 라이브러리 임포트

import pandas as pd
import re
import time
import requests
from datetime import datetime
from datetime import date
import pandas as pd
from bs4 import BeautifulSoup
import json

In [2]:
### 헤더, 함수 정의

headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Connection": "keep-alive",
        "Cache-Control": "max-age=0",
        "sec-ch-ua-mobile": "?0",
        "DNT": "1",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "ko-KR,ko;q=0.9"
    }

def get_soup(url, time_sleep=0, max_retries=600):
    try:
        if max_retries <= 0:
            print("[최대 재시도 횟수 초과 : get_soup()]")
            return None
        with requests.Session() as session:
            response = session.get(url, headers=headers)
            time.sleep(time_sleep)
        soup = BeautifulSoup(response.text, "html.parser")
        if len(soup) == 0:  # 불러오는데 실패하면
            print(f"[soup 로딩 실패, 반복] [get_soup(time_sleep=1)]")
            soup = get_soup(url, 1)
    except Exception as e:
        print(f"[오류 발생, 반복] [get_soup(time_sleep=1)] ", e)
        soup = get_soup(url, 1, max_retries-1)
    return soup

In [5]:
# 함수 2
# 전처리 함수 : 덱 타입
def preprocess_deck_type(deck_type):
    processed_deck_type = deck_type.lower()  # 소문자화
    processed_deck_type = re.sub(r'[^a-zA-Z ]', '', processed_deck_type)  # 영어 알파벳(대소문자)과 공백만 남기고 모두 제거
    return processed_deck_type.replace(' ', '_')  # 공백문자를 _로 변경

# 시간측정 데코레이터
def time_check(func):
  def wrapper(*args, **kwargs):
    start_time = datetime.now().replace(microsecond=0)  # 시작 시각
    print(f"함수 {func.__name__} 실행 시작 시간: {start_time}")
    
    result = func(*args, **kwargs) # 함수 실행

    end_time = datetime.now().replace(microsecond=0)  # 종료 시각
    print(f"함수 {func.__name__} 실행 종료 시간: {end_time}")
    print(f"함수 {func.__name__} 실행 시간: {end_time - start_time}")

    return result
  return wrapper

# tourney_data 에 1개의 tourney 메타 정보 누적
def crawl_one_tourney_meta(tourney_url, tourney_data, str_progress):
    print("[start] crawl_one_tourney_meta()")
    tourney_meta_url = tourney_url + "/metagame"
    soup = get_soup(tourney_meta_url)    # bs4로 html 가져오기
    tourney_deck_type_list = []
    min_deck_count = 1              # 최소 덱 타입 수

    # 정보 추출 : 덱 타입
    meta_tr_list = soup.select('table.meta tr')[1:]
    for tr in meta_tr_list:
        deck_count = int(tr.select('td')[1].text)
        if deck_count < min_deck_count:
            continue
        deck_type = preprocess_deck_type(tr.select('td')[2].text)
        deck_url = limitlesstcg_url_base + tr.select('td')[2].find('a')['href']
        # print(deck_count, deck_type, deck_url)
    
        # DB에 덱타입 추가 : 기존 정보가 있는지 확인
        if deck_type not in tourney_data:
            tourney_data[deck_type] = {}
        
        # df에 저장
        row = [deck_type, deck_count, deck_url]
        tourney_deck_type_list.append(row)
    
    # df : 덱 타입 (중복 덱타입 존재함)
    df_tourney_deck_type_list  = pd.DataFrame(tourney_deck_type_list, columns=["deck_type", "deck_count", "deck_url"])

    # 덱 타입 별 승/패 수 누적
    for index, row in df_tourney_deck_type_list.iterrows():
        print(f"Tourney {str_progress} Deck Type [{index+1}/{len(df_tourney_deck_type_list)}] {row['deck_type']}")
        matchup_deck_type = row['deck_type']
        matchup_url = row['deck_url'] + "/matchups"
        matchup_soup = get_soup(matchup_url)
    
        ### 정보 추출 : 덱 별 매치업 통계
        matchup_tr_list = matchup_soup.select('table.striped tr')[1:-1]
        
        for tr in matchup_tr_list:
            # 정보 추출
            opponent_deck_type = tr.select('td')[1].text.replace(' ', '_')
            win_count = int(tr.select('td')[2].text.split(' - ')[0])
            lose_count = int(tr.select('td')[2].text.split(' - ')[1])
            # print(opponent_deck_type, win_count, lose_count)
        
            # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
            try:
                tourney_data[matchup_deck_type][opponent_deck_type]
            except: 
                tourney_data[matchup_deck_type][opponent_deck_type] = {"win_count": 0, "lose_count": 0}
        
            # 3. 승리/패배 횟수 누적
            tourney_data[matchup_deck_type][opponent_deck_type]["win_count"] += win_count
            tourney_data[matchup_deck_type][opponent_deck_type]["lose_count"] += lose_count
    
            # print(tourney_data[matchup_deck_type])
    
    print("[end] crawl_one_tourney_meta()")
    
    return tourney_data

@time_check
def crawl_tourney_meta(tourney_result_file_name):
    # 1. 파일 불러오기
    df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
    df.info()

    # tourney_meta 선언
    tourney_meta = {}

    # 2. 모든 url에 대해 meta 정보 수집
    for index, row in df.iterrows():
        str_progress = f"[{index+1}/{len(df)}]"
        print(f"Tourney Meta {str_progress} {row['url']}")
        tourney_meta = crawl_one_tourney_meta(row['url'], tourney_meta, str_progress)

    # JSON 파일로 저장 
    meta_file_name = tourney_result_file_name.replace('csv', 'json').replace('result', 'meta')
    with open("./tourney_meta/" + meta_file_name, "w") as f:
        json.dump(tourney_meta, f, indent=4)  # indent=4는 가독성을 위해 들여쓰기


In [3]:
### 파일 불러오기
tourney_result_file_name = "tourney_result_250123_131739.csv"
df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
df.info()

FileNotFoundError: [Errno 2] No such file or directory: './tourney_result/tourney_result_250123_131739.csv'

# 1. 실험 : 하나의 tourney 대상으로 메타 분석

In [3]:
### 저장할 데이터베이스 초기화 : 2중 딕셔너리로 저장할 예정
tourney_data = {}

In [4]:
### 샘플 토너먼트 url 설정
# tourney_meta_url = "https://play.limitlesstcg.com/tournament/6779bb0bed2c78098c6b086b/metagame"
tourney_meta_url = "https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame"
limitlesstcg_url_base = "https://play.limitlesstcg.com"

## step a) 아키타입 별 상대 승률

In [22]:
### 전처리 함수 : 덱 타입
def preprocess_deck_type(deck_type):
    processed_deck_type = deck_type.lower()  # 소문자화
    processed_deck_type = re.sub(r'[^a-zA-Z ]', '', processed_deck_type)  # 영어 알파벳(대소문자)과 공백만 남기고 모두 제거
    return processed_deck_type.replace(' ', '_')  # 공백문자를 _로 변경

### bs4로 html 가져오기
soup = get_soup(tourney_meta_url)

### 정보 추출 : 덱 타입
tourney_deck_type_list = []
min_deck_count = 1  # 최소 덱 타입 수

meta_tr_list = soup.select('table.meta tr')
for tr in meta_tr_list[1:]:
    # 정보 추출
    deck_count = int(tr.select('td')[1].text)
    if deck_count < min_deck_count:
        continue
    deck_type = preprocess_deck_type(tr.select('td')[2].text)
    deck_url = limitlesstcg_url_base + tr.select('td')[2].find('a')['href']
    print(deck_count, deck_type, deck_url)

    # DB에 덱타입 추가 : 기존 정보가 있는지 확인
    if deck_type not in tourney_data:
        tourney_data[deck_type] = {}
    
    # df에 저장
    row = [deck_type, deck_count, deck_url]
    tourney_deck_type_list.append(row)

# df : 덱 타입 - 중복 덱타입 존재함.
df_tourney_deck_type_list  = pd.DataFrame(tourney_deck_type_list, columns=["deck_type", "deck_count", "deck_url"])

328 mewtwo_ex_gardevoir https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/mewtwo-ex-gardevoir-a1
237 pikachu_ex_zebstrika https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/pikachu-ex-zebstrika-a1
175 gyarados_ex_greninja https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/gyarados-ex-a1a-greninja-a1
103 exeggutor_ex_celebi_ex https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/exeggutor-ex-a1-celebi-ex-a1a
73 charizard_ex_moltres_ex https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/charizard-ex-moltres-ex-a1
63 celebi_ex_serperior https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/celebi-ex-serperior-a1a
52 charizard_ex_arcanine_ex https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/charizard-ex-arcanine-ex-a1
35 aerodactyl_ex_primeape https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/aerodactyl-

In [336]:
len(tourney_data)

251

In [337]:
len(tourney_deck_type_list)  
df_tourney_deck_type_list

,deck_type,deck_count,deck_url
0,mewtwo_ex_gardevoir,328,https://play.limitlesstcg.com/tournament/67830...
1,pikachu_ex_zebstrika,237,https://play.limitlesstcg.com/tournament/67830...
2,gyarados_ex_greninja,175,https://play.limitlesstcg.com/tournament/67830...
3,exeggutor_ex_celebi_ex,103,https://play.limitlesstcg.com/tournament/67830...
4,charizard_ex_moltres_ex,73,https://play.limitlesstcg.com/tournament/67830...
...,...,...,...
163,aerodactyl_ex_weezing,1,https://play.limitlesstcg.com/tournament/67830...
164,primeape_kabutops,1,https://play.limitlesstcg.com/tournament/67830...
165,moltres_ex_volcarona,1,https://play.limitlesstcg.com/tournament/67830...
166,zapdos_ex_raichu,1,https://play.limitlesstcg.com/tournament/67830...


In [318]:
### 덱 타입 필터링 
# min_deck_count = 300
# df_tourney_deck_type_filtered = df_tourney_deck_type_list[df_tourney_deck_type_list['deck_count'] >= min_deck_count]
# df_tourney_deck_type_filtered.tail()

,deck_type,deck_count,deck_url
0,mewtwo_ex_gardevoir,302,https://play.limitlesstcg.com/tournament/6779b...


In [338]:
### 덱 타입 별 

for index, row in df_tourney_deck_type_list.iterrows():
    print(f"index : {index}")
    print(row['deck_type'])
    matchup_deck_type = row['deck_type']
    matchup_url = row['deck_url'] + "/matchups"
    matchup_soup = get_soup(matchup_url)

    ### 정보 추출 : 덱 별 매치업 통계
    matchup_tr_list = matchup_soup.select('table.striped tr')[1:-1]
    
    for tr in matchup_tr_list:
        # 정보 추출
        opponent_deck_type = tr.select('td')[1].text.replace(' ', '_')
        win_count = int(tr.select('td')[2].text.split(' - ')[0])
        lose_count = int(tr.select('td')[2].text.split(' - ')[1])
        print(opponent_deck_type, win_count, lose_count)
    
        # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
        try:
            tourney_data[matchup_deck_type][opponent_deck_type]
        except: 
            tourney_data[matchup_deck_type][opponent_deck_type] = {"win_count": 0, "lose_count": 0}
    
        # 3. 승리/패배 횟수 누적
        tourney_data[matchup_deck_type][opponent_deck_type]["win_count"] += win_count
        tourney_data[matchup_deck_type][opponent_deck_type]["lose_count"] += lose_count

        # print(tourney_data[matchup_deck_type])

print("[done] ")

index : 0
mewtwo_ex_gardevoir
Blastoise_ex_Starmie_ex 2 0
Alakazam_Mew_ex 2 0
Zebstrika_Pikachu_ex 2 0
Wigglytuff_ex_Exeggutor_ex 2 0
Dragonite_Zebstrika 2 0
Machamp_ex_Aerodactyl_ex 2 0
Mewtwo_ex_Alakazam 2 0
Gyarados_ex_Kabutops 2 0
Greninja_Hitmonlee 1 0
Golbat_Mew_ex 1 0
Marowak_ex_Sandslash 1 0
Beheeyem_Mew_ex 1 0
Moltres_ex_Rapidash 1 0
Arcanine_ex_Salazzle 1 0
Machamp_ex_Druddigon 1 0
Alakazam_Jynx 1 0
Golem_Primeape 1 0
Gardevoir_Mewtwo 1 0
Gengar_ex_Druddigon 1 0
Alakazam_Clefable 1 0
Greninja_Melmetal 1 0
Golem_Farfetch'd 1 0
Aerodactyl_ex_Primeape 1 0
Mew_ex_Greninja 1 0
Machamp_ex_Mew_ex 1 0
Raichu_Zebstrika 1 0
Mew_ex_Golem 1 0
Alakazam_Florges 1 0
Machamp_ex_Primeape 1 0
Wigglytuff_ex_Weezing 1 0
Gyarados_ex_Chatot 1 0
Arcanine_ex_Greninja 1 0
Exeggutor_ex_Butterfree 1 0
Moltres_ex_Volcarona 1 0
Blastoise_ex_Druddigon 1 0
Machamp_ex_Hitmonlee 1 0
Golem_Druddigon 14 2
Pidgeot_ex_Hitmonlee 5 1
Ninetales_Rapidash 4 1
Charizard_ex_Moltres_ex 44 17
Aerodactyl_ex_Primeape 24 10

In [339]:
tourney_data

{'mewtwo_ex_gardevoir': {'Golem': {'win_count': 7, 'lose_count': 2},
  'Alakazam_Beheeyem': {'win_count': 4, 'lose_count': 0},
  'Exeggutor_ex_Greninja': {'win_count': 4, 'lose_count': 2},
  'Pidgeot_ex_Hitmonlee': {'win_count': 7, 'lose_count': 1},
  'Aerodactyl_ex_Onix': {'win_count': 3, 'lose_count': 1},
  'Articuno_ex_Mew_ex': {'win_count': 2, 'lose_count': 0},
  "Mew_ex_Farfetch'd": {'win_count': 4, 'lose_count': 1},
  'Greninja_Marshadow': {'win_count': 2, 'lose_count': 0},
  'Mew_ex_Alakazam': {'win_count': 2, 'lose_count': 0},
  'Moltres_ex_Rapidash': {'win_count': 3, 'lose_count': 0},
  'Marowak_ex_Aerodactyl_ex': {'win_count': 1, 'lose_count': 1},
  'Marowak_ex_Primeape': {'win_count': 2, 'lose_count': 0},
  'Marowak_ex_Golem': {'win_count': 1, 'lose_count': 0},
  "Golem_Farfetch'd": {'win_count': 2, 'lose_count': 0},
  'Machamp_ex_Mew_ex': {'win_count': 2, 'lose_count': 0},
  'Pidgeot_ex_Greninja': {'win_count': 1, 'lose_count': 0},
  'Mew_ex_Mewtwo_ex': {'win_count': 1, 'lo

In [328]:
# ### 테스트용 (위에 덧댈거임)
# matchup_deck_type = 'Mewtwo_ex_Gardevoir'
# matchup_url = "https://play.limitlesstcg.com/tournament/6779bb0bed2c78098c6b086b/metagame/mewtwo-ex-gardevoir-a1" + "/matchups"
# matchup_soup = get_soup(matchup_url)

# ### 정보 추출 : 덱 별 매치업 통계
# tr_list = soup.select('table.striped tr')[1:-1]

# for tr in tr_list:
#     # 정보 추출
#     opponent_deck_type = tr.select('td')[1].text.replace(' ', '_')
#     win_count = int(tr.select('td')[2].text.split(' - ')[0])
#     lose_count = int(tr.select('td')[2].text.split(' - ')[1])
#     print(opponent_deck_type, win_count, lose_count)

#     # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
#     if opponent_deck_type not in tourney_data[deck_type][opponent_deck_type]:
#         tourney_data[matchup_deck_type][opponent_deck_type] = {"승리": 0, "패배": 0}

#     # 3. 승리/패배 횟수 누적
#     tourney_data[matchup_deck_type][opponent_deck_type]["승리"] += win_count
#     tourney_data[matchup_deck_type][opponent_deck_type]["패배"] += lose_count

In [331]:
backup_tourney_data = tourney_data
backup_tourney_data

{'mewtwo_ex_gardevoir': {'Golem': {'win_count': 4, 'lose_count': 0},
  'Alakazam_Beheeyem': {'win_count': 4, 'lose_count': 0},
  'Exeggutor_ex_Greninja': {'win_count': 3, 'lose_count': 0},
  'Pidgeot_ex_Hitmonlee': {'win_count': 2, 'lose_count': 0},
  'Aerodactyl_ex_Onix': {'win_count': 2, 'lose_count': 0},
  'Articuno_ex_Mew_ex': {'win_count': 2, 'lose_count': 0},
  "Mew_ex_Farfetch'd": {'win_count': 2, 'lose_count': 0},
  'Greninja_Marshadow': {'win_count': 2, 'lose_count': 0},
  'Mew_ex_Alakazam': {'win_count': 2, 'lose_count': 0},
  'Moltres_ex_Rapidash': {'win_count': 2, 'lose_count': 0},
  'Marowak_ex_Aerodactyl_ex': {'win_count': 1, 'lose_count': 0},
  'Marowak_ex_Primeape': {'win_count': 2, 'lose_count': 0},
  'Marowak_ex_Golem': {'win_count': 1, 'lose_count': 0},
  "Golem_Farfetch'd": {'win_count': 1, 'lose_count': 0},
  'Machamp_ex_Mew_ex': {'win_count': 1, 'lose_count': 0},
  'Pidgeot_ex_Greninja': {'win_count': 1, 'lose_count': 0},
  'Mew_ex_Mewtwo_ex': {'win_count': 1, 'lo

## step a-2) 함수화
- 하나의 tourney에서 meta 정보를 불러온다.
- 기존의 tourney_data 에 정보를 누적시킨다.

In [74]:
# 전처리 함수 : 덱 타입
def preprocess_deck_type(deck_type):
    processed_deck_type = deck_type.lower()  # 소문자화
    processed_deck_type = re.sub(r'[^a-zA-Z ]', '', processed_deck_type)  # 영어 알파벳(대소문자)과 공백만 남기고 모두 제거
    return processed_deck_type.replace(' ', '_')  # 공백문자를 _로 변경

# 시간측정 데코레이터
def time_check(func):
  def wrapper(*args, **kwargs):
    start_time = datetime.now().replace(microsecond=0)  # 시작 시각
    print(f"함수 {func.__name__} 실행 시작 시간: {start_time}")
    
    result = func(*args, **kwargs) # 함수 실행

    end_time = datetime.now().replace(microsecond=0)  # 종료 시각
    print(f"함수 {func.__name__} 실행 종료 시간: {end_time}")
    print(f"함수 {func.__name__} 실행 시간: {end_time - start_time}")

    return result
  return wrapper

# tourney_data 에 1개의 tourney 메타 정보 누적
def crawl_one_tourney_meta(tourney_url, tourney_data, str_progress):
    print("[start] crawl_one_tourney_meta()")
    tourney_meta_url = tourney_url + "/metagame"
    soup = get_soup(tourney_meta_url)    # bs4로 html 가져오기
    tourney_deck_type_list = []
    min_deck_count = 1              # 최소 덱 타입 수

    # 정보 추출 : 덱 타입
    meta_tr_list = soup.select('table.meta tr')[1:]
    for tr in meta_tr_list:
        deck_count = int(tr.select('td')[1].text)
        if deck_count < min_deck_count:
            continue
        deck_type = preprocess_deck_type(tr.select('td')[2].text)
        deck_url = limitlesstcg_url_base + tr.select('td')[2].find('a')['href']
        # print(deck_count, deck_type, deck_url)
    
        # DB에 덱타입 추가 : 기존 정보가 있는지 확인
        if deck_type not in tourney_data:
            tourney_data[deck_type] = {}
        
        # df에 저장
        row = [deck_type, deck_count, deck_url]
        tourney_deck_type_list.append(row)
    
    # df : 덱 타입 (중복 덱타입 존재함)
    df_tourney_deck_type_list  = pd.DataFrame(tourney_deck_type_list, columns=["deck_type", "deck_count", "deck_url"])

    # 덱 타입 별 승/패 수 누적
    for index, row in df_tourney_deck_type_list.iterrows():
        print(f"Tourney {str_progress} Deck Type [{index+1}/{len(df_tourney_deck_type_list)}] {row['deck_type']}")
        matchup_deck_type = row['deck_type']
        matchup_url = row['deck_url'] + "/matchups"
        matchup_soup = get_soup(matchup_url)
    
        ### 정보 추출 : 덱 별 매치업 통계
        matchup_tr_list = matchup_soup.select('table.striped tr')[1:-1]
        
        for tr in matchup_tr_list:
            # 정보 추출
            opponent_deck_type = tr.select('td')[1].text.replace(' ', '_')
            win_count = int(tr.select('td')[2].text.split(' - ')[0])
            lose_count = int(tr.select('td')[2].text.split(' - ')[1])
            # print(opponent_deck_type, win_count, lose_count)
        
            # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
            try:
                tourney_data[matchup_deck_type][opponent_deck_type]
            except: 
                tourney_data[matchup_deck_type][opponent_deck_type] = {"win_count": 0, "lose_count": 0}
        
            # 3. 승리/패배 횟수 누적
            tourney_data[matchup_deck_type][opponent_deck_type]["win_count"] += win_count
            tourney_data[matchup_deck_type][opponent_deck_type]["lose_count"] += lose_count
    
            # print(tourney_data[matchup_deck_type])
    
    print("[end] crawl_one_tourney_meta()")
    
    return tourney_data

## step b) 20장 덱 유형 별 vs 아키타입 승률

## step c) 아키타입 별 승률 높은 20장 덱 유형 분석 (궁극의 덱이 무엇일까)

# 2. tourney_result.csv 받아서 다수의 토너먼트에 대해 실행 + 파일로 저장

## 2-a. 2개의 tourney에 대해 테스트

In [7]:
### 파일 불러오기
tourney_result_file_name = "tourney_result_250123_131739.csv"
df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          2 non-null      object
 1   tourney_name  2 non-null      object
 2   organizer     2 non-null      object
 3   players       2 non-null      int64 
 4   winner        2 non-null      object
 5   country       2 non-null      object
 6   url           2 non-null      object
dtypes: int64(1), object(6)
memory usage: 240.0+ bytes


In [16]:
# copy 데이터프레임 만들고, 값 확인
df_tourney_result = df
df_tourney_result['url'][0]

'https://play.limitlesstcg.com/tournament/678ae345010576265d112130'

In [18]:
# tourney_data 선언
tourney_data = {}

In [26]:
for index, row in df_tourney_result.iterrows():
    print(index, row['url'])
    tourney_data = crawl_one_tourney_meta(row['url'], tourney_data)

0 https://play.limitlesstcg.com/tournament/678ae345010576265d112130
step 1
16 mewtwo_ex_gardevoir https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/mewtwo-ex-gardevoir-a1
9 gyarados_ex_greninja https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/gyarados-ex-a1a-greninja-a1
6 pikachu_ex_zebstrika https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/pikachu-ex-zebstrika-a1
5 exeggutor_ex_celebi_ex https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/exeggutor-ex-a1-celebi-ex-a1a
5 celebi_ex_serperior https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/celebi-ex-serperior-a1a
4 aerodactyl_ex_primeape https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/aerodactyl-ex-primeape-a1a
4 starmie_ex_lumineon https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/starmie-ex-a1-lumineon-a1a
4 charizard_ex_moltres_ex https://play.limitlesstcg.co

In [28]:
tourney_data

{'mewtwo_ex_gardevoir': {'Gyarados_ex_Greninja': {'win_count': 5,
   'lose_count': 12},
  'Charizard_ex_Moltres_ex': {'win_count': 8, 'lose_count': 6},
  'Pidgeot_Arbok': {'win_count': 2, 'lose_count': 0},
  'Arbok_Weezing': {'win_count': 1, 'lose_count': 0},
  'Gengar_ex_Gengar': {'win_count': 1, 'lose_count': 0},
  'Arcanine_ex_Moltres_ex': {'win_count': 1, 'lose_count': 0},
  'Articuno_ex': {'win_count': 1, 'lose_count': 0},
  'Gyarados_ex_Vaporeon': {'win_count': 1, 'lose_count': 0},
  'Serperior_Celebi_ex': {'win_count': 2, 'lose_count': 1},
  'Celebi_ex_Serperior': {'win_count': 8, 'lose_count': 7},
  "Aerodactyl_ex_Farfetch'd": {'win_count': 1, 'lose_count': 1},
  'Gyarados_ex_Articuno_ex': {'win_count': 1, 'lose_count': 1},
  'Exeggutor_ex_Mew_ex': {'win_count': 2, 'lose_count': 1},
  'Mewtwo_ex_Gardevoir': {'win_count': 18, 'lose_count': 18},
  'Greninja_Tauros': {'win_count': 1, 'lose_count': 3},
  'Exeggutor_ex_Celebi_ex': {'win_count': 10, 'lose_count': 14},
  'Pikachu_ex_Z

In [29]:
backup_t_data = tourney_data
backup_t_data

{'mewtwo_ex_gardevoir': {'Gyarados_ex_Greninja': {'win_count': 5,
   'lose_count': 12},
  'Charizard_ex_Moltres_ex': {'win_count': 8, 'lose_count': 6},
  'Pidgeot_Arbok': {'win_count': 2, 'lose_count': 0},
  'Arbok_Weezing': {'win_count': 1, 'lose_count': 0},
  'Gengar_ex_Gengar': {'win_count': 1, 'lose_count': 0},
  'Arcanine_ex_Moltres_ex': {'win_count': 1, 'lose_count': 0},
  'Articuno_ex': {'win_count': 1, 'lose_count': 0},
  'Gyarados_ex_Vaporeon': {'win_count': 1, 'lose_count': 0},
  'Serperior_Celebi_ex': {'win_count': 2, 'lose_count': 1},
  'Celebi_ex_Serperior': {'win_count': 8, 'lose_count': 7},
  "Aerodactyl_ex_Farfetch'd": {'win_count': 1, 'lose_count': 1},
  'Gyarados_ex_Articuno_ex': {'win_count': 1, 'lose_count': 1},
  'Exeggutor_ex_Mew_ex': {'win_count': 2, 'lose_count': 1},
  'Mewtwo_ex_Gardevoir': {'win_count': 18, 'lose_count': 18},
  'Greninja_Tauros': {'win_count': 1, 'lose_count': 3},
  'Exeggutor_ex_Celebi_ex': {'win_count': 10, 'lose_count': 14},
  'Pikachu_ex_Z

In [44]:
# JSON 파일로 저장 
meta_file_name = "backup_t_data.json"
with open("./tourney_meta/" + meta_file_name, "w") as f:
    json.dump(backup_t_data, f, indent=4)  # indent=4는 가독성을 위해 들여쓰기

In [45]:
# JSON 파일에서 2중 딕셔너리 로드

with open("./tourney_meta/" + meta_file_name, "r") as f:
    loaded_data = json.load(f)

print(loaded_data)

{'mewtwo_ex_gardevoir': {'Gyarados_ex_Greninja': {'win_count': 5, 'lose_count': 12}, 'Charizard_ex_Moltres_ex': {'win_count': 8, 'lose_count': 6}, 'Pidgeot_Arbok': {'win_count': 2, 'lose_count': 0}, 'Arbok_Weezing': {'win_count': 1, 'lose_count': 0}, 'Gengar_ex_Gengar': {'win_count': 1, 'lose_count': 0}, 'Arcanine_ex_Moltres_ex': {'win_count': 1, 'lose_count': 0}, 'Articuno_ex': {'win_count': 1, 'lose_count': 0}, 'Gyarados_ex_Vaporeon': {'win_count': 1, 'lose_count': 0}, 'Serperior_Celebi_ex': {'win_count': 2, 'lose_count': 1}, 'Celebi_ex_Serperior': {'win_count': 8, 'lose_count': 7}, "Aerodactyl_ex_Farfetch'd": {'win_count': 1, 'lose_count': 1}, 'Gyarados_ex_Articuno_ex': {'win_count': 1, 'lose_count': 1}, 'Exeggutor_ex_Mew_ex': {'win_count': 2, 'lose_count': 1}, 'Mewtwo_ex_Gardevoir': {'win_count': 18, 'lose_count': 18}, 'Greninja_Tauros': {'win_count': 1, 'lose_count': 3}, 'Exeggutor_ex_Celebi_ex': {'win_count': 10, 'lose_count': 14}, 'Pikachu_ex_Zebstrika': {'win_count': 10, 'lose_

In [39]:
type(loaded_data['mewtwo_ex_gardevoir']['Gyarados_ex_Greninja']['win_count'])

int

## 2-b. 함수화

In [75]:
@time_check
def crawl_tourney_meta(tourney_result_file_name):
    # 1. 파일 불러오기
    df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
    df.info()

    # tourney_meta 선언
    tourney_meta = {}

    # 2. 모든 url에 대해 meta 정보 수집
    for index, row in df.iterrows():
        str_progress = f"[{index+1}/{len(df)}]"
        print(f"Tourney Meta {str_progress} {row['url']}")
        tourney_meta = crawl_one_tourney_meta(row['url'], tourney_meta, str_progress)

    # JSON 파일로 저장 
    meta_file_name = tourney_result_file_name.replace('csv', 'json').replace('result', 'meta')
    with open("./tourney_meta/" + meta_file_name, "w") as f:
        json.dump(tourney_meta, f, indent=4)  # indent=4는 가독성을 위해 들여쓰기


In [58]:
tourney_result_file_name = "tourney_result_250123_131739.csv"
crawl_tourney_meta(tourney_result_file_name)


함수 crawl_tourney_meta 실행 시작 시간: 2025-01-23 18:03:55
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          2 non-null      object
 1   tourney_name  2 non-null      object
 2   organizer     2 non-null      object
 3   players       2 non-null      int64 
 4   winner        2 non-null      object
 5   country       2 non-null      object
 6   url           2 non-null      object
dtypes: int64(1), object(6)
memory usage: 240.0+ bytes
tourney_meta [1/2] https://play.limitlesstcg.com/tournament/678ae345010576265d112130
[start] crawl_one_tourney_meta()
16 mewtwo_ex_gardevoir https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/mewtwo-ex-gardevoir-a1
9 gyarados_ex_greninja https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/gyarados-ex-a1a-greninja-a1
6 pikachu_ex_zebstrika https://play.limitlesstcg.c

## 2-c. 모든 tourney에 대해 실행

In [62]:
### 파일 불러오기
tourney_result_file_name = "tourney_result.csv"
df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          119 non-null    object
 1   tourney_name  119 non-null    object
 2   organizer     119 non-null    object
 3   players       119 non-null    int64 
 4   winner        119 non-null    object
 5   country       119 non-null    object
 6   url           119 non-null    object
dtypes: int64(1), object(6)
memory usage: 6.6+ KB


In [63]:
df.head(30)

,date,tourney_name,organizer,players,winner,country,url
0,2025-01-23,The Dark League Wednesday #4,The Dark League,184,Nana7x7,Japan,https://play.limitlesstcg.com/tournament/678d6...
1,2025-01-23,Pocket League #9,Mr-Gatito,180,ssEisen,Portugal,https://play.limitlesstcg.com/tournament/67885...
2,2025-01-22,Last Minute STANDARD FAST ROUNDS,Pokemon Downunder,163,Ekkinx,Spain,https://play.limitlesstcg.com/tournament/678f6...
3,2025-01-21,Monday Meta Tournament #3,Pocket Champions,205,toa,Unknown,https://play.limitlesstcg.com/tournament/678e0...
4,2025-01-21,Budew Takeover (pocket) pop up,Turtle Squad,142,mishzra,Philippines,https://play.limitlesstcg.com/tournament/678c3...
5,2025-01-20,Hora Da Batalha: Champions #1,Hora da Batalha,159,8PCheese_youtube,Japan,https://play.limitlesstcg.com/tournament/678aa...
6,2025-01-19,BEC Exeggutive Series! 2/8 1000$ USD,Blazing Exeggutor Crew,533,kiritan,Japan,https://play.limitlesstcg.com/tournament/67794...
7,2025-01-19,#4 Paldea Pro League 10 USD,just_samuele,164,rwindio,Brazil,https://play.limitlesstcg.com/tournament/67829...
8,2025-01-19,Chaos League Pocket Saturdays #4,Chaos League,214,ShangHigh,Korea,https://play.limitlesstcg.com/tournament/676d7...
9,2025-01-19,Ursiiday's Pocket Weekly #12 400 USD Prize,Ursiiday,1610,bob83727,Taiwan,https://play.limitlesstcg.com/tournament/67830...


In [76]:
tourney_result_file_name = "tourney_result.csv"
crawl_tourney_meta(tourney_result_file_name)

함수 crawl_tourney_meta 실행 시작 시간: 2025-01-23 20:15:45
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          119 non-null    object
 1   tourney_name  119 non-null    object
 2   organizer     119 non-null    object
 3   players       119 non-null    int64 
 4   winner        119 non-null    object
 5   country       119 non-null    object
 6   url           119 non-null    object
dtypes: int64(1), object(6)
memory usage: 6.6+ KB
Tourney Meta [1/119] https://play.limitlesstcg.com/tournament/678d6fa5010576265d1183d8
[start] crawl_one_tourney_meta()
Tourney [1/119] Deck Type [1/48] mewtwo_ex_gardevoir
Tourney [1/119] Deck Type [2/48] gyarados_ex_greninja
Tourney [1/119] Deck Type [3/48] exeggutor_ex_celebi_ex
Tourney [1/119] Deck Type [4/48] celebi_ex_serperior
Tourney [1/119] Deck Type [5/48] pikachu_ex_zebstrika
Tourney [1/119] Deck Type 

In [7]:
# JSON 파일에서 2중 딕셔너리 로드
tourney_result_file_name = "tourney_result.csv"
meta_file_name = tourney_result_file_name.replace('csv', 'json').replace('result', 'meta')
with open("./tourney_meta/" + meta_file_name, "r") as f:
    loaded_meta_data = json.load(f)

print(loaded_meta_data)

{'mewtwo_ex_gardevoir': {'Charizard_ex_Arcanine_ex': {'win_count': 538, 'lose_count': 344}, 'Greninja_Hitmonlee': {'win_count': 7, 'lose_count': 5}, 'Muk_Weezing': {'win_count': 11, 'lose_count': 10}, 'Machamp_ex_Aerodactyl_ex': {'win_count': 36, 'lose_count': 6}, 'Alakazam_Jynx': {'win_count': 5, 'lose_count': 0}, 'Moltres_ex_Centiskorch': {'win_count': 15, 'lose_count': 9}, 'Greninja_Gyarados_ex': {'win_count': 35, 'lose_count': 55}, 'Starmie_ex_Articuno_ex': {'win_count': 182, 'lose_count': 134}, 'Starmie_ex_Lumineon': {'win_count': 105, 'lose_count': 83}, 'Aerodactyl_ex_Primeape': {'win_count': 599, 'lose_count': 278}, "Starmie_ex_Farfetch'd": {'win_count': 1, 'lose_count': 0}, 'Machamp_ex': {'win_count': 16, 'lose_count': 5}, "Greninja_Farfetch'd": {'win_count': 79, 'lose_count': 74}, 'Exeggutor_ex_Mew_ex': {'win_count': 129, 'lose_count': 157}, 'Pikachu_ex_Raichu': {'win_count': 466, 'lose_count': 438}, 'Greninja_Mew_ex': {'win_count': 189, 'lose_count': 249}, 'Pikachu_ex_Zebstri

In [16]:
def calculate_wins_losses_from_meta(meta_data):
    total_wins = 0
    total_losses = 0
    for deck_type, matchups in meta_data.items():
        for opponent_deck, results in matchups.items():
            total_wins += results["win_count"]
            total_losses += results["lose_count"]
    return total_wins, total_losses

total_wins, total_losses = calculate_wins_losses_from_meta(loaded_meta_data)

In [19]:
total_wins, total_losses  # 기권패를 제외한 통계라, loss가 적음

(90599, 85495)

In [20]:
# 총 88047 건의 매치
(total_wins + total_losses)/2

88047.0

In [46]:
def make_df_from_meta(meta_data):
    temp_list = []
    for deck_type, matchups in meta_data.items():
        win_count = 0
        loss_count = 0
        for opponent_deck, results in matchups.items():
            win_count += results["win_count"]
            loss_count += results["lose_count"]

        matchup_count = win_count + loss_count 
        if matchup_count == 0:
            continue
        else:
            win_rate = win_count / (win_count + loss_count)
            temp_row = [deck_type, matchup_count, win_count, loss_count, win_rate]
            temp_list.append(temp_row)
        
    df = pd.DataFrame(temp_list, columns=["deck_type", "matchup_count", "loss_count", "loss_count", "win_rate"])
    return df

In [47]:
df_meta = make_df_from_meta(loaded_meta_data)
df_meta

,deck_type,matchup_count,loss_count,loss_count,win_rate
0,mewtwo_ex_gardevoir,32779,16801,15978,0.512554
1,gyarados_ex_greninja,23094,12367,10727,0.535507
2,exeggutor_ex_celebi_ex,9154,5002,4152,0.546428
3,celebi_ex_serperior,9956,4849,5107,0.487043
4,pikachu_ex_zebstrika,16819,8910,7909,0.529758
...,...,...,...,...,...
794,sandslash_dugtrio,6,4,2,0.666667
795,sandslash_grapploct,2,0,2,0.000000
796,articuno_ex_moltres_ex,3,1,2,0.333333
797,starmie_ex_pidgeot,4,2,2,0.500000


In [39]:
# 덱 타입 중복 없음
df_meta['deck_type'].nunique()

212

In [40]:
df_meta.head(30)

,deck_type,wins,losses,win_rate
0,mewtwo_ex_gardevoir,16801,15978,0.512554
1,gyarados_ex_greninja,12367,10727,0.535507
2,exeggutor_ex_celebi_ex,5002,4152,0.546428
3,celebi_ex_serperior,4849,5107,0.487043
4,pikachu_ex_zebstrika,8910,7909,0.529758
5,charizard_ex_moltres_ex,2869,2893,0.497917
6,exeggutor_ex_mew_ex,777,698,0.526780
7,arcanine_ex_moltres_ex,2274,1981,0.534430
8,greninja_mew_ex,1283,1153,0.526683
9,charizard_ex_arcanine_ex,2586,2261,0.533526


In [41]:
df_meta_sorted = df_meta.sort_values(by='win_rate', ascending=False)
df_meta_sorted.head(100)

,deck_type,wins,losses,win_rate
210,greninja_mewtwo_ex,36,16,0.692308
146,aerodactyl_ex_greninja,81,37,0.686441
165,marowak_ex_farfetchd,30,14,0.681818
179,gengar_ex_beheeyem,21,10,0.677419
189,wigglytuff_ex_melmetal,23,11,0.676471
...,...,...,...,...
75,marowak_ex_aerodactyl_ex,184,173,0.515406
45,marowak_ex_primeape,101,95,0.515306
92,golem_primeape,37,35,0.513889
126,raichu_magneton,59,56,0.513043


In [48]:
df_meta_sorted = df_meta.sort_values(by='matchup_count', ascending=False)


,deck_type,matchup_count,loss_count,loss_count,win_rate
0,mewtwo_ex_gardevoir,32779,16801,15978,0.512554
1,gyarados_ex_greninja,23094,12367,10727,0.535507
4,pikachu_ex_zebstrika,16819,8910,7909,0.529758
3,celebi_ex_serperior,9956,4849,5107,0.487043
2,exeggutor_ex_celebi_ex,9154,5002,4152,0.546428
...,...,...,...,...,...
171,pidgeot_ex,111,63,48,0.567568
117,moltres_ex_ninetales,111,52,59,0.468468
84,mew_ex_melmetal,108,52,56,0.481481
193,exeggutor_ex_wigglytuff_ex,108,58,50,0.537037


In [49]:
df_meta_sorted.head(100).sort_values(by='win_rate', ascending=False)


,deck_type,matchup_count,loss_count,loss_count,win_rate
241,aerodactyl_ex_greninja,118,81,37,0.686441
324,primeape_mew_ex,203,131,72,0.645320
55,greninja_beheeyem,121,73,48,0.603306
68,exeggutor_ex_exeggutor,273,164,109,0.600733
35,wigglytuff_ex_greninja,114,68,46,0.596491
...,...,...,...,...,...
157,dragonite_mew_ex,264,109,155,0.412879
115,mew_ex_alakazam,183,74,109,0.404372
174,alakazam_beheeyem,169,68,101,0.402367
94,alakazam_weezing,135,52,83,0.385185
